# Introdução

# Dependências

In [43]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install requests

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import requests
import tweepy

import json
import os

# Coleta

## Previsões

In [3]:
predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
filename = 'image_predictions.tsv'
if not os.path.exists(filename):
    response = requests.get(predictions_url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [42]:
df_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
df_predictions_clean = df_predictions.copy()

## Tweets

In [43]:
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_clean = df_tweets.copy()

## Contagem de retweets e favoritos

In [7]:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_CONSUMER_SECRET']
access_token = os.environ['TWITTER_ACCESS_TOKEN']
access_token_secret = os.environ['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitter_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
def get_retweet_fav_counts(row):
    tweet_id = row['tweet_id']
    try:
        tweet_status = twitter_api.get_status(row['tweet_id'], tweet_mode='extended')
        return (tweet_id, tweet_status.retweet_count, tweet_status.favorite_count)
    except tweepy.TweepError:
        return (tweet_id, None, None)

In [6]:
filename = 'tweet_counts.csv'
if not os.path.exists(filename):
    tweet_counts = df_tweets_clean.apply(lambda row: get_retweet_fav_counts(row), axis=1)
    df_tweet_counts = pd.DataFrame.from_records(list(tweet_counts), columns=['tweet_id', 'retweet_count', 'favorite_count'])
    df_tweet_counts.to_csv(filename, index=False)
else:
    df_tweet_counts = pd.read_csv(filename)

In [7]:
df_tweet_counts_clean = df_tweet_counts.copy()

# Avaliação

## Estrutura do DataFrame de previsões

In [44]:
df_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Estrutura do DataFrame de tweets

In [8]:
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Identificação de Retweets

In [9]:
df_retweets = df_tweets_clean[df_tweets_clean.in_reply_to_status_id.notnull()]
df_retweets.shape[0]

78

## Denominador com valor abaixo de 10

In [10]:
df_tweets_clean[df_tweets_clean.rating_denominator < 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None


## Contagem de retweets e favoritos faltantes

In [33]:
df_retweet_count_null = df_tweet_counts_clean[(df_tweet_counts_clean.retweet_count.isnull()) | (df_tweet_counts_clean.favorite_count.isnull())]
df_retweet_count_null.shape[0]

15

## Tweets sem referência do nome do animal

In [93]:
df_no_name = df_tweets_clean[df_tweets_clean.name == 'None']
df_no_name.shape[0]

745

# Documentação dos Problemas de Qualidade

## Problemas de Qualidade

1. Tipo das colunas "in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id" deve ser alterado de float64 para int64 (seção "Estrutura do DataFrame de tweets")
2. Tipo das colunas "timestamp" e "retweeted_status_timestamp" deve ser alterado de string (object) para datetime
3. 78 tweets são retweets (seção "Identificação de Retweets")
4. 3 tweets com denominador abaixo de 10 (seção "Denominador com valor abaixo de 10")
5. 745 tweets não contém o nome do animal a qual se referem (seção "Tweets sem referência do nome do animal")

## Problemas de Arrumação

1. DataFrame com as previsões contem 9 colunas relacionadas aos valores de previsão, onde apenas 3 colunas devem ser utilizadas para normalizar estes valores (previsão, intervalo de confiança e valor booleano indicando se a previsão é uma raça de cachorro) (seção "Estrutura do DataFrame de previsões")
2. DataFrame com os tweets contem 4 colunas relacionadas ao "estágio" do animal, podendo ser normalizada para apenas uma coluna "estágio" (seção "Estrutura do DataFrame de tweets")

# Limpeza

## Contagem de retweets e favoritos

In [62]:
df_tweet_counts_clean['retweet_count'] = df_tweet_counts_clean['retweet_count'].fillna(0).astype(int)
df_tweet_counts_clean['favorite_count'] = df_tweet_counts_clean['favorite_count'].fillna(0).astype(int)
df_tweet_counts_clean[(df_tweet_counts_clean.retweet_count.isnull()) | (df_tweet_counts_clean.favorite_count.isnull())]

,tweet_id,retweet_count,favorite_count


## Ajuste das predições

In [ ]:
df_predictions_1 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p1", "p1_conf", "p1_dog"]].copy()
df_predictions_2 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p2", "p2_conf", "p2_dog"]].copy()
df_predictions_3 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p3", "p3_conf", "p3_dog"]].copy()

df_predictions_1.rename(columns={'p1': 'prediction', 'p1_conf': 'confidence_interval', 'p1_dog': 'is_dog_race'}, inplace=True)
df_predictions_2.rename(columns={'p2': 'prediction', 'p2_conf': 'confidence_interval', 'p2_dog': 'is_dog_race'}, inplace=True)
df_predictions_3.rename(columns={'p3': 'prediction', 'p3_conf': 'confidence_interval', 'p3_dog': 'is_dog_race'}, inplace=True)

df_predictions_clean = pd.concat([df_predictions_1, df_predictions_2, df_predictions_3])
df_predictions_clean.shape[0]